In [1]:
project_root_path = '/home/bob/projects/Web3HackerNetwork'

### Load the Commit Statistics
For this clustering we examined 39 GitHub projects containing a total of 2,629 commits. The projects all included the Fluence Marine library. Code for Fluence is generally written in React and Rust.

In [2]:
import sys
bob_lib_path = project_root_path + "/sandbox/python/bob/"
sys.path.append(bob_lib_path)

import commit_stat_engineering
import json

dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

### Load Labels

In [3]:
import pandas as pd

data = pd.read_csv(project_root_path + '/sandbox/data/bob/commit_labeling.tsv', sep='\t')
data

,Unnamed: 0,is_bulk,is_standard,is_minor,is_artifact,is_javascript,is_rust
0,a7503dcb63af05b1381c8fbc7d1aa989ce61d605,1,0,0,0,0.0,0.0
1,a91521059eeac8bfaa6b81a0333413b0015c8863,1,0,0,0,0.0,0.0
2,e5e0d59835a849eb9c43b2cd20b25046e1ddab3e,1,0,0,0,0.0,0.0
3,cf3c18490aaee04cdee75a3d22d67e8877b8abe7,1,0,0,0,0.0,0.0
4,f1907df36197dc3d7be452aab244d7652c4c4181,1,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...
64,e2560c165fbd05e6bb41a7e9166f5b2283f15de4,0,1,0,0,1.0,0.0
65,fbf6540ed115934fb9641e9743b67608c978a942,0,1,0,0,1.0,0.0
66,e55ed70029f41ad1fa59a2cb89b88ebea96bb9f2,0,0,1,0,1.0,0.0
67,e19efe724f7e0a1054a62a7c3b064882e6048365,0,1,0,0,0.0,0.0


### Link Labels to Stats

In [43]:
import numpy as np
def make_x(commit_stats):
    return [
        np.log1p(commit_stats['binFiles']),
        np.log1p(commit_stats['textFiles']),
        np.log1p(commit_stats['textLines']),
        np.log1p(commit_stats['javascriptLines']),
        np.log1p(commit_stats['rustLines']),
        np.log1p(commit_stats['jsonLines']),
        #np.log1p(commit_stats['imgFiles']),
        np.log1p(commit_stats['lockLines']),
        np.log1p(commit_stats['htmlLines'])
    ]


In [67]:
stats = {}
for commit in all_stats:
    stats[commit['commit']] = commit

X = []
Y = []
a = 0
rows = [row for row in data.to_numpy().tolist() if row[4] != 1]
for row in rows: #data.to_numpy() if row['is_artifact'] != 1:
    commit_id = row[0]
    row_labels = row[slice(1, 4)]
    x = make_x(stats[commit_id])
    #print(x)
    #print(a)
    X.append(make_x(stats[commit_id]))
    Y.append(row_labels)#.tolist())
    a += 1
#print(a)

#print(Y[slice(2)])
#print(Y[0][0])
#print(X[slice(2)])

In [68]:
from sklearn.datasets import make_multilabel_classification

sample_X, sample_y = make_multilabel_classification(n_classes=3, random_state=0)
print(sample_y[slice(2)])
print(Y[slice(2)])

[[0 1 0]
 [0 1 0]]
[[1, 0, 0], [1, 0, 0]]


In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

print(len(X))
print(len(Y))

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X, Y)
score = clf.score(X, Y)
Y_pred = clf.predict(X)
# print(score)
# print(Y_pred)

65
65


In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.33, random_state=3)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 6), random_state = 1)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
# Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

0.9090909090909091


In [9]:
#print(X_test)
#print(Y_pred)
pairs = []
for i in range(0,len(X_test)):
    #print(Y_pred[i].tolist())
    pairs.append(Y_pred[i].tolist() + X_test[i])
# print(pairs)

print(len(X_train))
import pandas as pd
pd.DataFrame(pairs).to_csv("/home/bob/test.tsv", sep="\t")

46
